# Importing Libraries

In [25]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from pathlib import Path

# Data Preprocessing

In [26]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=128,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=True,

)

Found 70295 files belonging to 38 classes.


In [27]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,

)

Found 17572 files belonging to 38 classes.


In [28]:
training_set

<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>

# Building Model

In [29]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD


## MobileNet Model

In [30]:
import h5py
print(h5py.__version__)

3.9.0


In [31]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet')

### Fine-Tuning

In [32]:
# Add custom classification layers on top of the pre-trained model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)  # Add dropout to reduce overfitting
x = Dense(38, activation='softmax')(x)  # Adjust the output size based on your task

In [33]:
base_model.trainable = True

set_trainable = False

for layer in base_model.layers:
    if layer.name == 'block_14_expand':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [34]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_376 (Conv2D)            (None, 63, 63, 32)   864         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_376 (Batch  (None, 63, 63, 32)  96          ['conv2d_376[0][0]']             
 Normalization)                                                                                   
                                                                                       

### Add custom classifcation layer

In [35]:
# # Normalize
# def process(image,label):
#     image = tf.cast(image/255. ,tf.float32)
#     return image,label

# train_ds = training_set.map(process)
# validation_ds = validation_set.map(process)

In [36]:
# Create the fine-tuned model
InceptModel = Model(inputs=base_model.input, outputs=x)
optimizer_SGD = SGD(learning_rate=1e-4, momentum=0.9)

# Compile the model
InceptModel.compile(
    
    optimizer=optimizer_SGD,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [37]:
InceptModel.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_376 (Conv2D)            (None, 63, 63, 32)   864         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_376 (Batch  (None, 63, 63, 32)  96          ['conv2d_376[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [38]:
history_Incept = InceptModel.fit(training_set,epochs=10,validation_data=validation_set)

Epoch 1/10
550/550 [==============================] - 844s 2s/step - loss: 4.8226 - accuracy: 0.0373 - val_loss: 3.5823 - val_accuracy: 0.0525
Epoch 2/10
550/550 [==============================] - 671s 1s/step - loss: 3.6132 - accuracy: 0.0398 - val_loss: 3.5492 - val_accuracy: 0.0625
Epoch 3/10
550/550 [==============================] - 677s 1s/step - loss: 3.5793 - accuracy: 0.0515 - val_loss: 3.5412 - val_accuracy: 0.0657
Epoch 4/10
550/550 [==============================] - 674s 1s/step - loss: 3.5696 - accuracy: 0.0548 - val_loss: 3.5258 - val_accuracy: 0.0734
Epoch 5/10
550/550 [==============================] - 667s 1s/step - loss: 3.5603 - accuracy: 0.0556 - val_loss: 3.5146 - val_accuracy: 0.0697
Epoch 6/10
550/550 [==============================] - 666s 1s/step - loss: 3.5523 - accuracy: 0.0577 - val_loss: 3.5165 - val_accuracy: 0.0763
Epoch 7/10
550/550 [==============================] - 669s 1s/step - loss: 3.5445 - accuracy: 0.0602 - val_loss: 3.4925 - val_accuracy: 0.0754

In [39]:
#Training set Accuracy
train_loss, train_acc = InceptModel.evaluate(training_set)
print('Training accuracy:', train_acc)

550/550 [==============================] - 520s 945ms/step - loss: 3.5028 - accuracy: 0.0742
Training accuracy: 0.07421580702066422


In [40]:
#Validation set Accuracy
val_loss, val_acc = InceptModel.evaluate(validation_set)
print('Validation accuracy:', val_acc)

550/550 [==============================] - 152s 277ms/step - loss: 3.5100 - accuracy: 0.0730
Validation accuracy: 0.07301388680934906


In [ ]:
InceptModel.save('InceptModel.keras')

In [42]:
history_Incept.history #Return Dictionary of history

{'loss': [4.822634220123291,
  3.61321759223938,
  3.5793099403381348,
  3.5696442127227783,
  3.5602669715881348,
  3.5522823333740234,
  3.5444819927215576,
  3.5311243534088135,
  3.52079439163208,
  3.506989002227783],
 'accuracy': [0.037342626601457596,
  0.039789456874132156,
  0.05152571201324463,
  0.05476918816566467,
  0.055622734129428864,
  0.05767124146223068,
  0.0602034293115139,
  0.06273561716079712,
  0.06398747861385345,
  0.06573724746704102],
 'val_loss': [3.5822787284851074,
  3.5491538047790527,
  3.541217803955078,
  3.5258047580718994,
  3.514646530151367,
  3.5165340900421143,
  3.492487668991089,
  3.507291316986084,
  3.483358144760132,
  3.5100181102752686],
 'val_accuracy': [0.05246983841061592,
  0.06254268437623978,
  0.06572957336902618,
  0.07341224700212479,
  0.06971318274736404,
  0.07631459087133408,
  0.07540405541658401,
  0.07016845047473907,
  0.07688368111848831,
  0.07301388680934906]}

In [43]:
#Recording History in json
import json
with open('history_Incept.json','w') as f:
  json.dump(history_Incept.history,f)


In [44]:
print(history_Incept.history.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


## Loading Model

In [45]:
print(history_Incept)

In [46]:
epochs = [i for i in range(1,11)]
plt.plot(epochs,history_Incept['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs,history_Incept['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel('No. of Epochs')
plt.title('Visualization of Accuracy Result')
plt.legend()
plt.show()

TypeError: 'History' object is not subscriptable

## Some other metrics to evaluate our Model

In [ ]:
class_name = validation_set.class_names

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=1,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
y_pred = InceptModel.predict(test_set)
predicted_categories = tf.argmax(y_pred, axis=1)

In [ ]:
true_categories = tf.concat([y for x, y in test_set], axis=0)
Y_true = tf.argmax(true_categories, axis=1)

In [ ]:
Y_true

In [ ]:
predicted_categories

In [ ]:
import sklearn.metrics

from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(Y_true,predicted_categories)

In [ ]:
# Precision Recall Fscore
print(classification_report(Y_true,predicted_categories,target_names=class_name))

## Conflict Matrix Visualization

In [ ]:
plt.figure(figsize=(40, 40))
sns.heatmap(cm,annot=True,annot_kws={"size": 10})

plt.xlabel('Predicted Class',fontsize = 20)
plt.ylabel('Actual Class',fontsize = 20)
plt.title('Plant Disease Prediction Confusion Matrix',fontsize = 25)
plt.show()